# Exchange Step

**Goal**: Implement the exchange step on a simple example using `numpy`. If you're not familiar with `numpy`, there's a basic tutorial on `moodle`.

You are given the following LP in canonical form:

$$\begin{equation}
\begin{array}
\\max \  z=&4 x_1 &+ &9 x_2, &       &  \\
  subject \  to:    &x_1     &+ &4 x_2   & \leq  &40\\
         &2x_1    &+ & x_2    & \leq  &42\\
         &1.5x_1  &+ &3x_2    & \leq  &36\\
         &x_1     &  &        & \geq  &0 \\
         &        &  &x_2     & \geq  &0 \\
\end{array}
\end{equation}$$


Your task is to implement a function that performs a pivoting step on the element in the $i$-th row and $j$-th column of the tableau $T$.<br>
To get the tableau $T$ you first need to convert the LP into standard form:


$$\begin{equation}
\begin{array}
\\max \  z=       &    & &    & &    & &4 x_1   &+ &9 x_2,  &       &  \\
  subject \  to:  &y_1 &+&    & &    & &x_1     &+ &4 x_2   & =     &40\\
                  &    & &y_2 &+&    & &2x_1    &+ & x_2    & =     &42\\
                  &    & &    & &y_3 &+&1.5x_1  &+ &3x_2    & =     &36\\
                  &    & &    & &    & &x_1     &  &        & \geq  &0 \\
                  &    & &    & &    & &        &  &x_2     & \geq  &0 \\
                  &y_1 & &    & &    & &        &  &        & \geq  &0 \\
                  &    & &y_2 & &    & &        &  &        & \geq  &0 \\
                  &    & &    & &y_3 & &        &  &        & \geq  &0 \\
\end{array}
\end{equation}$$

From this you can construct the tableau:

$$\begin{equation}
\begin{array}{l|rrrrr|r}
  & y_1 & y_2 & y_3 & x_1 & x_2 & 1\\
  \hline
 z & 0 & 0 & 0 & -4 & -9 & 0\\
 \hline
  & 1 & 0 & 0 & 1 & 4 & 40\\
  & 0 & 1 & 0 & 2 & 1 & 42\\
  & 0 & 0 & 1 & 1.5 & 3 & 36
\end{array}
\end{equation}$$

In [1]:
import numpy as np
#Define Tableau
T = np.matrix([
    [0, 0, 0,  -4, -9, 0 ],
    [1, 0, 0,   1,  4, 40],
    [0, 1, 0,   2,  1, 42],
    [0, 0, 1, 1.5,  3, 36]
])

## Task 1

### Exchange Step

The entries of the new tableau are given in the lecture notes and the computation goes as follows:

$$\begin{equation}
\begin{array}
((i) & a'_{il} = \frac{a_{il}}{a_{ik}} & l=1,...,n+m &\\
(ii) & a'_{jl} = a_{jl} - \frac{ a_{jk}a_{il}}{a_{ik}} & l=1,...,n+m & j \neq i, j=1,...,m
\end{array}
\end{equation}$$

In [2]:
def pivot(T,i,k):
    #Create a copy of the tableau
    T_pivot = np.copy(T)
    
    #Step (i)
    #We divide the i-th row elementwise by the value of a_ik
    T_pivot[i,:] = T[i,:] / T[i,k]
    
    #Step (ii)
    #Note: for-loops can be interchanged
    #We iterate through each column (l = 1,...,n+m)
    for l in range(0,T.shape[1]):
        #In the column we iterate through each row, but skip row i (j=/=i,j=1,...,m)
        for j in range(0,T.shape[0]):
            if(j!=i):
                #We compute the new value
                T_pivot[j,l] = T[j,l] - T[j,k] * T[i,l] / T[i,k]
    
    return T_pivot

### Testing
To see if your implementation works compare it with the result you get doing it by hand.

In [3]:
#Testing with a pivot at position (3,5).
pivot(T,2,4)

array([[   0. ,    9. ,    0. ,   14. ,    0. ,  378. ],
       [   1. ,   -4. ,    0. ,   -7. ,    0. , -128. ],
       [   0. ,    1. ,    0. ,    2. ,    1. ,   42. ],
       [   0. ,   -3. ,    1. ,   -4.5,    0. ,  -90. ]])

If we perform the calculations by hand, pivoting on the element (3,5) leads to the following tableau:

$$\begin{equation}
\begin{array}{l|rrrrr|r}
  & y_1 & y_2 & y_3 & x_1 & x_2 & 1\\
  \hline
z & 0 & 9 & 0 & 14 & 0 & 378\\
\hline
  & 1 & -4 & 0 & -7 & 0 & -128\\
  & 0 & 1 & 0 & 2 & 1 & 42\\
  & 0 & -3 & 1 & -4.5 & 0 & -90
\end{array}
\end{equation}$$

We subtracted the third row 4 times from the second row and 3 times from the forth row.

## Task 2

Solve the problem using the pivoting function you implemented and compare your result with the result you get with `pulp`.

### Solution via `pulp`

In [4]:
import pulp

#Create the LP
mylp = pulp.LpProblem("ExchangeStep", pulp.LpMaximize)

#Create the variables
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.LpContinuous)

#Add the objective function
mylp += 4*x1 + 9*x2

#Add the constraints
mylp += x1 + 4*x2 <= 40
mylp += 2*x1 + x2 <= 42
mylp += 1.5*x1 + 3*x2 <= 36

mylp.solve()
print("Solution obtained with pulp:")
#Print out the optimal objective value of my_lp
print(mylp.objective.value())
#Print out optimal vertex.
for v in mylp.variables():
    print(str(v) + " = " + str(v.value()))

Solution obtained with pulp:
104.0
x1 = 8.0
x2 = 8.0


### Manual solution

We first pivot on the element (2,5) and then on the element (4,4). Note that both pivots are valid (Theorem 2.4 in the lecture notes).<br>
After we have done the two steps, we see that we only have positive values the first row. Therefore any further exchange step would only lower the objective value.<br>
We have found an optimal solution that is consistent with the one obtained with pulp.

In [5]:
T_2 = pivot(T,1,4)
print("Tableau after the first pivot:")
print(T_2)

T_3 = pivot(T_2,3,3)
print("Tableau after the second pivot:")
print(T_3)

print("Solution obtained manually:")
print(T_3[0,5])
print("x1 = " + str(T_3[3,5]))
print("x2 = " + str(T_3[1,5]))

Tableau after the first pivot:
[[ 2.25  0.    0.   -1.75  0.   90.  ]
 [ 0.25  0.    0.    0.25  1.   10.  ]
 [-0.25  1.    0.    1.75  0.   32.  ]
 [-0.75  0.    1.    0.75  0.    6.  ]]
Tableau after the second pivot:
[[  0.5          0.           2.33333333   0.           0.
  104.        ]
 [  0.5          0.          -0.33333333   0.           1.
    8.        ]
 [  1.5          1.          -2.33333333   0.           0.
   18.        ]
 [ -1.           0.           1.33333333   1.           0.
    8.        ]]
Solution obtained manually:
104.0
x1 = 8.0
x2 = 8.0
